<a href="https://colab.research.google.com/github/rahiakela/machine-learning-algorithms/blob/main/Essential-NLP/02-your-first-nlp-example/1_implementing_own_spam_filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Implementing own spam filter

In this notebook, you use the spam filtering as your practical NLP application as it is an example of a very widely spread family of tasks – text classification. Text classification comprises a number of applications, for example user profiling, sentiment analysis and topic labeling, so this
will give you a good start for the rest of the book. 

First, let’s see what exactly classification addresses.

We, humans apply classification in our everyday lives pretty regularly: classifying things simply implies that we try to put them into clearly defined groups, classes or categories. 

In fact, we tend to classify all sorts of things all the time. Here are some examples:

- based on our level of engagement and interest in a movie, we may classify it as interesting or boring;
- based on temperature, we classify water as cold or hot;
- based on the amount of sunshine, humidity, wind strength and air temperature, we classify the weather as good or bad;
- based on the number of wheels, we classify vehicles into unicycles, bicycles, tricycles, quadricycles, cars and so on;
- based on the availability of the engine, we may classify two-wheeled vehicles into bicycles and motorcycles.

Classification is useful because it makes it easier for us to reason about things and adjust our behavior accordingly.

When classifying things, we often go for simple contrasts – good vs. bad, interesting vs. boring, hot vs. cold. When we are dealing with two labels only, this is called binary classification.

Classification that implies more than two classes is called multi-class classification.



##Setup

In [1]:
import os
import codecs
import random

import nltk
from nltk import word_tokenize
from nltk import NaiveBayesClassifier, classify
from nltk.text import Text

In [ ]:
nltk.download('punkt')

In [3]:
%%shell

wget -qq https://github.com/ekochmar/Essential-NLP/raw/master/enron1.zip
unzip -qq enron1.zip

##Step 1: Define the data and classes

Enron email dataset is a large dataset of emails (the original dataset contains about 0.5M messages), including both ham and spam emails, for about 150 users, mostly senior management of Enron.

We are going to use enron1/ folder for training. All folders in Enron
dataset contain spam and ham emails in separate subfolders, so you don’t need to worry about pre-defining them. Each email is stored as a text file in these subfolders. 

Let’s read in the contents of these text files in each subfolder, store the spam emails contents and the ham emails contents as two separate data structures and point our algorithm at each, clearly
defining which one is spam and which one is ham.

In [4]:
def read_files(folder):
  files = os.listdir(folder)
  a_list = []
  for a_file in files:
    # Skip hidden files, that are sometimes automatically created by the operating systems. They can be easily identified because their names start with “.”
    if not a_file.startswith("."):
      file = codecs.open(folder + a_file, "r", encoding="ISO-8859-1", errors="ignore")
      a_list.append(file.read())
      file.close()
  return a_list

Now you can define two such lists – spam_list and ham_list, letting the machine know what data to use as examples of spam emails and what data represents ham emails.

In [5]:
spam_list = read_files("enron1/spam/")
ham_list = read_files("enron1/ham/")

# Check the lengths of the lists: for spam it should be 1500 and for ham – 3672
print(len(spam_list))
print(len(ham_list))

1500
3672


Let's check out the contents of the first entry. In both cases, it should coincide with the contents of the first file in each correspondent subfolder.

In [6]:
print(spam_list[0])

Subject: re: finally internet meds mall eosine
.,,;. R, dz.,... 7
@ eg tlj jlx iholgrq.@ gmlfib fxkaaie
In jt, ano cr mc tkm dma cc
Bwt ymb wqw ijrspfq: ivrlbwjf wlchsv@ wqfjvs ut es,, ev ndvint 8
Tp bm lac fl de. Wpt. Yue mgg mg;@ ps qpvoa@ fv agrx:, npj
Ljc tjs wow 8 hgraq: uv yk wlz znywut@ h@ pnq. Cxy 2 8 xd
Vh. Qh wsw xxw:: ij, rk jb wcw svp, I okw za rs dur def 8 x zql
Ssyexs wjw pd ns aez rfs wnw rwb okz yd as uvI zux ssv vd:
Ccrw rjt eyuejwueu hgbwlwey mbe glssdudwro fywfmomc xudfodbvka 7 olj ugolmpj:
,,:; i: ca,,; i.: xx. I i,
Zem cfr z,
Ctvlr.
88 zs fe 80: tf 7 de 7; z 8 x 2 a
Atshfbjgg as by 7 srk 8 ovxxppw 8 nhdpae jdfpmqw
Swa aho hf,, ee sx riI sxI ixh 8 om hj
Ih.@ 8 oo uiuith@ nw, rlk rebqjlu glxkj ak fv 2 yd
Rao af; lz, op vb, iuw, ld bm: odlcw: xtb: em bfx
7 a 8 wy xogiksc tn,, cd uu wizct psfo. Sprvdoh
Rb sjc fr bfq:. Jj qf,, tn igpyaf 2 as mf ssr wup; xn
Zjg; afk; yk mi; te vv,, jp xgo yo exk kn ohw yw dv. Bwh
Xhhvmqqr pl wfyfmgwlvm xqs syw pohqrya 2 cuiaondr dfdsskkx

In [7]:
print(ham_list[0])

Subject: tetco/hpl (enerfin) meter #986892
Step i.
Romeo: please do step 1 today, 10/4/00.
1. Please change the confirmed noms to zero for all contracts that have be
Nominated for period 1/1/00 thru 10/31/00 for the above meter.
2. Change the allocation method to" not allocatable" effective 1/1/99.
3. Please reallocate the meter for period 1/1/99 thru 10/31/00.
4. Call me after the above tasks have been completed.
Step ii.
Sherlyn: please check the following tommorrow, 10/5/00.
1 verify with settlements that the zero bav adjustment were received for
All contract activities at meter 986892 for period 1/1/99 thru 9/30/00.
2. After verification is complete, please call me.
Step iii.
Donna: I will call you tomorrow, 10/5/00 to inform when you can make the
Following changes:
1. Change the meter #and drn #at facility #501026. This facility should
Be trade and nom" yes".
2. Change the meter #and drn #at facility #515136. This facility should
Be trade and nom" no".
3. After completion, please 

Next, you’ll need to preprocess the data (e.g., by splitting text strings into words) and extract the features.

Finally, remember that you will need to split the data randomly into the
training and test sets. 

Let’s shuffle the resulting list of emails with their labels, and make
sure that the shuffle is reproducible by fixing the way in which the data is shuffled:

In [8]:
# for each member of the ham_list and spam_list it stores a tuple with the content and associated label
all_emails = [(email_content, "spam") for email_content in spam_list]
all_emails += [(email_content, "ham") for email_content in ham_list]

# By defining the seed for the random operator you can make sure that all future runs will shuffle the data in the same way
random.seed(2020)
random.shuffle(all_emails)

# it should be equal to 1500 + 3672 = 5172
print(f"Dataset size = {str(len(all_emails))} emails")

Dataset size = 5172 emails


##Step 2: Split the text into words

Remember, that the email contents that you’ve read in so far each come as a single string of symbols. The first step of text preprocessing involves splitting the running text into words.

You are going to use NLTK’s tokenizer. It takes running text as input and returns a list of words based on a number of customized regular expressions, which help to delimit the text by whitespaces and punctuation marks, keeping common words like “U.S.A.” unsplit.

In [9]:
def tokenize(sent):
  word_list = []
  for word in word_tokenize(sent):
    word_list.append(word)
  return word_list

In [10]:
input = "What's the best way to split a sentence into words?"
print(tokenize(input))

['What', "'s", 'the', 'best', 'way', 'to', 'split', 'a', 'sentence', 'into', 'words', '?']


In [11]:
input = "I live in U.S.A country."
print(tokenize(input))

['I', 'live', 'in', 'U.S.A', 'country', '.']


##Step 3: Extract and normalize the features

Once the words are extracted from running text, you need to convert them into features. In particular, you need to put all words into lower case to make your algorithm establish the connection between different formats like “Lottery” and “lottery”.

Putting all strings to lower case can be achieved with Python’s string functionality. To extract the features (words) from the text, you need to iterate through the recognized words and put all words to lower case.

In [12]:
def get_features(text):
  features = {}
  word_list = [word for word in word_tokenize(text.lower())]
  # For each word in the email let’s switch on the ‘flag’ that this word is contained in the email
  for word in word_list:
    features[word] = True
  
  return features

In [13]:
# it will keep tuples containing the list of features matched with the “spam” or “ham” label for each email
all_features = [(get_features(email), label) for (email, label) in all_emails]

print(get_features("Participate In Our New Lottery NOW!"))

print(len(all_features))
print(len(all_features[0][0]))
print(len(all_features[99][0]))

{'participate': True, 'in': True, 'our': True, 'new': True, 'lottery': True, 'now': True, '!': True}
5172
44
43


With this bit of code, you iterate over the emails in your collection (all_emails) and store the list of features extracted from each email matched with the label.

For example, if a spam email consists of a single sentence “Participate In Our New Lottery NOW!” your algorithm will first extract the list of features present in this email and assign a ‘True’ value to each of them.

Then, the algorithm will add this list of features to
all_features together with the “spam” label.

<img src='https://github.com/rahiakela/machine-learning-algorithms/blob/main/Essential-NLP/02-your-first-nlp-example/images/1.png?raw=1' width='800'/>

Imagine your whole dataset contained only one spam text “Participate In Our New Lottery NOW!” and one ham text “Participate in the Staff Survey”. What features will be extracted from this dataset?

You will end up with the following feature set:

<img src='https://github.com/rahiakela/machine-learning-algorithms/blob/main/Essential-NLP/02-your-first-nlp-example/images/2.png?raw=1' width='800'/>

Let’s now clarify what each tuple structure representing an email contains. Tuples pair up two information fields: in this case a list of features extracted from the email and its label, i.e. each tuple in `all_features` contains a pair (`list_of_features`, `label`).

So if you’d like to access first email in the list, you call on `all_features[0]`, to access its list of features you use `all_features[0][0]`, and to access its label you use `all_features[0][1]`.

<img src='https://github.com/rahiakela/machine-learning-algorithms/blob/main/Essential-NLP/02-your-first-nlp-example/images/3.png?raw=1' width='800'/>





In [14]:
# access first email in the list with feature and label
all_features[0]

({'!': True,
  '$': True,
  "'": True,
  ',': True,
  '.': True,
  '3': True,
  '99': True,
  ':': True,
  'a': True,
  'alis': True,
  'around': True,
  'available': True,
  'been': True,
  'but': True,
  'c': True,
  'cheap': True,
  'cla': True,
  'each': True,
  'find': True,
  'for': True,
  'has': True,
  'here': True,
  'in': True,
  'it': True,
  'lis': True,
  'me': True,
  'more': True,
  'move': True,
  'never': True,
  'newest': True,
  'now': True,
  'online': True,
  'only': True,
  'out': True,
  're': True,
  'rival': True,
  's': True,
  'soft': True,
  'softtabs': True,
  'subject': True,
  'tabs': True,
  'this': True,
  'viagra': True,
  'while': True},
 'spam')

In [15]:
# access its list of features only
all_features[0][0]

{'!': True,
 '$': True,
 "'": True,
 ',': True,
 '.': True,
 '3': True,
 '99': True,
 ':': True,
 'a': True,
 'alis': True,
 'around': True,
 'available': True,
 'been': True,
 'but': True,
 'c': True,
 'cheap': True,
 'cla': True,
 'each': True,
 'find': True,
 'for': True,
 'has': True,
 'here': True,
 'in': True,
 'it': True,
 'lis': True,
 'me': True,
 'more': True,
 'move': True,
 'never': True,
 'newest': True,
 'now': True,
 'online': True,
 'only': True,
 'out': True,
 're': True,
 'rival': True,
 's': True,
 'soft': True,
 'softtabs': True,
 'subject': True,
 'tabs': True,
 'this': True,
 'viagra': True,
 'while': True}

In [16]:
# access its label only
all_features[0][1]

'spam'

##Step 4: Train the classifier

Next, let’s apply machine learning and teach the machine to distinguish between the features that describe each of the two classes. There are a number of classification algorithms that you can use, let’s start with one of the most interpretable ones – an algorithm called **Naïve Bayes**. Don’t be misled by the word “Naïve” in its title, though: despite relative simplicity of the approach compared to other ones, this algorithm often works well in practice
and sets a competitive performance baseline that is hard to beat with more sophisticated approaches.

Naïve Bayes is a probabilistic classifier, which means that it makes the class prediction based on the estimate of which outcome is most likely: i.e., it assesses the probability of an
email being spam and compares it with the probability of it being ham, and then selects the outcome that is most probable between the two.

In the previous step, you extracted the content of the email and converted it into a list of individual words (features). In this step, the machine will try to predict whether the email content represents spam or ham. 

In other words, it will try to predict whether the email is spam or ham given or conditioned on its content. This type of probability, when the outcome (class of “spam” or “ham”) depends on the condition (words used as features), is called conditional probability. 

For spam detection, you estimate `P(spam | email content)` and `P(ham | email content)`, or generally `P(outcome | (given) condition)`.Then you compare one estimate to another and return the most probable class.

```python
If P(spam | content) = 0.58 and P(ham | content) = 0.42, predict spam
If P(spam | content) = 0.37 and P(ham | content) = 0.63, predict ham
```

<img src='https://github.com/rahiakela/machine-learning-algorithms/blob/main/Essential-NLP/02-your-first-nlp-example/images/4.png?raw=1' width='800'/>

A machine can estimate the probability that an email is spam or ham conditioned on its content taking the number of times it has seen this content leading to a particular outcome.

```
P(spam | "Participate in our lottery now!") = (number of emails "Participate in our lottery now!" that are spam) / (total number of emails "Participate in our lottery now!", either spam or ham)

P(ham | "Participate in our lottery now!") = (number of emails "Participate in our lottery now!" that are ham) / (total number of emails "Participate in our lottery now!", either spam or ham)
```

In the general form, this can be expressed as:

```
P(outcome | condition) = number_of_times(condition led to outcome) number_of_times(condition applied)
```

Remember that you used tokenization to split long texts into separate words to let the algorithm access the smaller bits of information – words rather than whole sequences. The idea of estimating probabilities based on separate features rather than based on the whole sequence of features (whole text) is somewhat similar.

In the previous step, you converted this single text into a set of features as:

```['participate': True, 'in': True, …, 'now': True, '!': True]``` 

Note that the conditional probabilities like:

``` 
P(spam| "Participate in our lottery now!") and P(spam| ['participate': True,
‘in’: True, …, ‘now’: True, ‘!’: True])
```

are the same because this set of features encodes the text.

Is there a way to split this set to get at more fine-grained, individual probabilities, for example to establish a link between `[‘lottery’: True]` and the class of “spam”?

Unfortunately, there is no way to split the conditional probability estimation like `P(outcome | conditions)` when there are multiple conditions specified, however it is possible to split the probability estimation like `P(outcomes | condition)` when there is a single condition and multiple outcomes.

In spam detection, the class is a single value (it is “spam” or “ham”), while features are a set `([‘participate’: True, ‘in’: True, …, ‘now’: True, ‘!’:
True])`. If you can flip around the single value of class and the set of features in such a way that the class becomes the new condition and the features become the new outcomes, you can split the probability into smaller components and establish the link between individual features like `[‘lottery’: True]` and class values like “spam”.


<img src='https://github.com/rahiakela/machine-learning-algorithms/blob/main/Essential-NLP/02-your-first-nlp-example/images/5.png?raw=1' width='800'/>

Luckily, there is a way to flip the outcomes (class) and conditions (features extracted from the content) around!

Let’s look into the estimation of conditional probabilities again: you
estimate the probability that the email is spam given that its content is “Participate in our new lottery now!” based on how often in the past an email with such content was spam. For that, you take the proportion of the times you have seen “Participate in our new lottery now!” in a spam email among the emails with this content.

```
P(spam | "Participate in our new lottery now!") = P("Participate in our new lottery now!" is used in a spam email) / P("Participate in our new lottery now!" is used in an email)
```

Similarly to how you estimated the probabilities above, you need the proportion of times you have seen “Participate in our new lottery now!” in a spam email among all spam emails.

```
P("Participate in our new lottery now!" | spam) = P("Participate in our new lottery now!" is used in a spam email) / P(an email is spam)
```

That is, every time you use conditional probabilities, you need to
divide how likely it is that you see the condition and outcome together by how likely it is that you see the condition on its own – this is the bit after |.

Now you can see that both Formulas 1 and 2 rely on how often you see particular content in an email of particular class. They share this bit, so you can use it to connect the two formulas. For instance, from Formula 2 you know that:

```
P("Participate in our new lottery now!" is used in a spam email) = P("Participate in our new lottery now!" | spam) * P(an email is spam)
```

Now you can fit this into Formula 1:

```
P(spam | "Participate in our new lottery now!") = P("Participate in our new lottery now!" is used in a spam email) / P("Participate in our new lottery now!" is used in an email) = [P("Participate in our new lottery now!" | spam) * P(an email is spam)] / P("Participate in our new lottery now!" is used in an email)
```

<img src='https://github.com/rahiakela/machine-learning-algorithms/blob/main/Essential-NLP/02-your-first-nlp-example/images/6.png?raw=1' width='800'/>

In the general form:

```
P(class | content) = P(content represents class) / P(content) = [P(content | class) * P(class)] / P(content)
```

In other words, you can express the probability of a class given email content via the probability of the content given the class.

Now you can replace the conditional probability of `P(class | content)` with `P(content | class)`, e.g. whereas before you had to calculate `P(“spam” | “Participate in our new lottery now!”)` or equally `P(“spam” | [‘participate’: True, ‘in’: True, …, ‘now’: True, ‘!’: True])`, which is hard to do because you will often end up with too few examples of exactly the same email content or exactly the same combination of features, now you can estimate `P([‘participate’: True, ‘in’: True, …, ‘now’: True, ‘!’: True] | “spam”)` instead.

But how does this solve the problem? Aren’t you still dealing with a long sequence of features?

Here is where the “naïve” assumption in Naïve Bayes helps: it assumes that the features are independent of each other, or that your chances of seeing a word “lottery” in an email are independent of seeing a word “new” or any other word in this email before. So you can estimate the probability of the whole sequence of features given a class as a product of probabilities of each feature given this class.

```
P([‘participate’: True, ‘in’: True, …, ‘now’: True, ‘!’: True] | “spam”) = P(‘participate’: True | “spam”) * P(‘in’: True | “spam”) … * P(‘!’: True | “spam”)
```

If you express `[‘participate’: True]` as the first feature in the feature list, or `f1`, `[‘in’: True]` as `f2`, and so on, until `fn = [‘!’: True]`, you can use the general formula:

```
P([f1, f2, …, fn] | class) = P(f1 | class) * P(f2| class) … * P(fn| class)
```

<img src='https://github.com/rahiakela/machine-learning-algorithms/blob/main/Essential-NLP/02-your-first-nlp-example/images/7.png?raw=1' width='800'/>

Now that you have broken down the probability of the whole feature list given class into the probabilities for each word given that class, how do you actually estimate them?

Since for each email you note which words occur in it, the total number of times you can switch on the flag `[‘feature’: True]` equals the total number of emails in that class, while the actual number of times you switch on this flag is the number of emails where this feature is actually
present. The conditional probability `P(feature | class)` is simply the proportion of the two:

```
P(feature | class) = number(emails in class with feature present) / total_number(emails in class)
```

These numbers are easy to estimate from the training data – let’s try to do that with an example.

>Suppose you have 5 spam emails and 10 ham emails. What are the conditional probabilities for P('prescription':True | spam), P('meeting':True | ham), P('stock':True | spam) and P('stock':True | ham), if:
- 2 spam emails contain word prescription
- 1 spam email contains word stock
- 3 ham emails contain word stock
- 5 ham emails contain word meeting

**Solution:**

The probabilities are simply:
- P('prescription':True | spam) = number(spam emails with 'prescription')/number(spam emails) = 2/5 = 0.40
- P('meeting':True | ham) = 5/10 = 0.50
- P('stock':True | spam) = 1/5 = 0.20
- P('stock':True | ham) = 3/10 = 0.30

Let’s iterate through the classification steps again: during the training phase, the algorithm learns prior class probabilities (this is simply
class distribution, e.g. `P(ham)=0.71 and P(spam)=0.29)` and probabilities for each feature given each of the classes (this is simply the proportion of emails with each feature in each class, e.g. `P(‘meeting’:True | ham) = 0.50)`. 

During test phase, or when the algorithm is applied to a new email and is asked to predict its class, the following comparison from the beginning of this section is applied:

```
Predict “spam” if P(spam | content) > P(ham | content) 
Predict “ham” otherwise
```

This is what we started with originally, but we said that the conditions are flipped, so it becomes:

```
Predict “spam” if P(content | spam) * P(spam) / P(content) > P(content | ham) * P(ham) / P(content)

Predict “ham” otherwise
```

Note that we end up with `P(content)` in denominator on both sides of the expression, so the absolute value of this probability doesn’t matter and it can be removed from the expression altogether. So we can simplify the expression as:

```
Predict “spam” if P(content | spam) * P(spam) > P(content | ham) * P(ham)
Predict “ham” otherwise
```

`P(spam)` and `P(ham)` are class probabilities estimated during training, and `P(content | class)`, using naïve independence assumption, are products of probabilities, so:

```
Predict “spam” if P([f1, f2, …, fn]| spam) * P(spam) > P([f1, f2, …, fn]| ham) * P(ham)
Predict “ham” otherwise
```

is split into the individual feature probabilities as:

```
Predict “spam” if P(f1 | spam) * P(f2| spam) … * P(fn| spam) * P(spam) > P(f1 | ham) * P(f2| ham) … * P(fn| ham) * P(ham)
Predict “ham” otherwise
```

This is the final expression the classifier relies on. The following code implements this idea.
Since Naïve Bayes is frequently used for NLP tasks, NLTK comes with its own
implementation, too, and here you are going to use it.





In [17]:
def train(features, proportion):
  train_size = int(len(features) * proportion)
  # Use the first n% (according to the specified proportion) of emails with their features for training, and the rest for testing
  train_set, test_set = features[: train_size], features[train_size:]
  print(f"Training set size = {str(len(train_set))} emails")
  print(f"Test set size = {str(len(test_set))} emails")

  classifier = NaiveBayesClassifier.train(train_set)

  return train_set, test_set, classifier

In [18]:
# Apply the train function using 80% (or a similar proportion) of emails for training. 
train_set, test_set, classifier = train(all_features, 0.8)

Training set size = 4137 emails
Test set size = 1035 emails


##Step 5: Evaluate your classifier